<!-- HTML file automatically generated from DocOnce source (https://github.com/doconce/doconce/)
doconce format html week10.do.txt --no_mako -->
<!-- dom:TITLE: March 27-31: Advanced machine learning and data analysis for the physical sciences -->

# March 27-31: Advanced machine learning and data analysis for the physical sciences
**Morten Hjorth-Jensen**, Department of Physics and Center for Computing in Science Education, University of Oslo, Norway and Department of Physics and Astronomy and Facility for Rare Isotope Beams, Michigan State University, East Lansing, Michigan, USA

Date: **March 27-31, 2023**

## Plans for the week March 27-31

**Deep generative models.**

1. Monte Carlo methods and structured probabilistic models for deep learning

2. Partition function and Boltzmann machines

3. Reading recommendation: Goodfellow et al chapters 16-18

4. [Video of lecture](https://youtu.be/)

5. [Whiteboard notes](https://github.com/CompPhysics/AdvancedMachineLearning/blob/main/doc/HandwrittenNotes/NotesMar292023.pdf)